## 5.0 测试模型性能
&emsp;由于作者代码问题，原代码并没有采用测试集的数据进行展示。

&emsp;所以首先要下载test的数据集和信息文件（这个在0号文件中会有简单说明），其次要修改`retinanet/dataloader.py`文件中的39行`self.coco      = COCO(os.path.join(self.root_dir, 'annotations', 'instances_' + self.set_name + '.json'))`，注释掉;接着去掉40行的注释`# self.coco      = COCO('coco/annotations/image_info_test2017.json')`。

&emsp;与train的代码一致，只能用调用py文件的方式运行，预测的结果会跳出窗口呈现。详细的代码注释在下一块中呈现。

In [2]:
!python visualize.py

CUDA available: True
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
/media/atara/WD SSD/PartTime/ObjectDetection/pytorch-retinanet/retinanet/model.py:52: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output contiguous tensor even if some of the inputs are in channels_last format. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/aten/src/ATen/native/TensorIterator.cpp:918.)
  P4_x = P5_upsampled_x + P4_x  # 与C5上采样后的特征相加
/media/atara/WD SSD/PartTime/ObjectDetection/pytorch-retinanet/retinanet/model.py:57: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output channels_last tensor even if some of the inputs are not in channels_last format. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/aten/src/ATen/native/TensorIterator.cpp:924.)
  P3_x = P3_x + P4_upsampled_x  # 与C4上采样后的特征相加
Elapsed time: 0.348475933074951

Elapsed time: 0.1474895477294922
person
car
car
car
car
car
car
car
car
car
Elapsed time: 0.12238383293151855
zebra
zebra
zebra
zebra
zebra
zebra
zebra
zebra
zebra
zebra
zebra
Elapsed time: 0.09568309783935547
person
frisbee
orange
Elapsed time: 0.14115262031555176
person
person
person
truck
truck
truck
Elapsed time: 0.1237785816192627
pizza
dining table
Elapsed time: 0.1412053108215332
person
person
person
person
person
person
person
boat
dog
Elapsed time: 0.13500761985778809
airplane
Elapsed time: 0.12853097915649414
person
person
person
person
bicycle
motorcycle
Elapsed time: 0.12243103981018066
person
person
person
person
person
person
sports ball
Elapsed time: 0.11190581321716309
person
cup
bed
dining table
Elapsed time: 0.13732242584228516
bottle
bottle
bottle
bottle
bottle
bottle
bottle
bottle
microwave
refrigerator
vase
vase
Elapsed time: 0.09868621826171875
person
Elapsed time: 0.11200881004333496
toilet
sink
Elapsed time: 0.15172839164733887
person
bed
Elapsed time: 0.1252269

In [ ]:
import numpy as np
import torchvision
import time
import os
import copy
import pdb
import time
import argparse

import sys
import cv2

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

from retinanet import model
from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, Augmenter, \
	UnNormalizer, Normalizer


assert torch.__version__.split('.')[0] == '1'

print('CUDA available: {}'.format(torch.cuda.is_available()))


def main(args=None):
	parser = argparse.ArgumentParser(description='Simple training script for training a RetinaNet network.')

	parser.add_argument('--dataset', help='Dataset type, must be one of csv or coco.', default='coco')  # 数据集的类型
	parser.add_argument('--coco_path', help='Path to COCO directory', default='coco')  # 数据的路径
	parser.add_argument('--csv_classes', help='Path to file containing class list (see readme)')  # 自定义数据集的类别文件路径
	parser.add_argument('--csv_val', help='Path to file containing validation annotations (optional, see readme)')  # 自定义数据集val集合的路径

	parser.add_argument('--model', help='Path to model (.pt) file.', default='pre_model/coco_resnet_50_map_0_335_state_dict.pt')  # 预训练好模型的路径

	parser = parser.parse_args(args)

	if parser.dataset == 'coco':  # 读入test的数据，注意`set_name='test2017'`要正确，代表了测试图片所在的路径
		dataset_val = CocoDataset(parser.coco_path, set_name='test2017', transform=transforms.Compose([Normalizer(), Resizer()]))
	elif parser.dataset == 'csv':
		dataset_val = CSVDataset(train_file=parser.csv_train, class_list=parser.csv_classes, transform=transforms.Compose([Normalizer(), Resizer()]))
	else:
		raise ValueError('Dataset type not understood (must be csv or coco), exiting.')

	# 该部分与训练部分一致
	sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
	dataloader_val = DataLoader(dataset_val, num_workers=1, collate_fn=collater, batch_sampler=sampler_val)

	# Create the model  新建模型，读入参数（原作者代码有误，得这么改才行）
	retinanet = model.resnet50(num_classes=dataset_val.num_classes(),)
	retinanet.load_state_dict(torch.load(parser.model))

	use_gpu = True

	if use_gpu:  # 有GPU则用
		if torch.cuda.is_available():
			retinanet = retinanet.cuda()

	if torch.cuda.is_available():
		retinanet = torch.nn.DataParallel(retinanet).cuda()
	else:
		retinanet = torch.nn.DataParallel(retinanet)

	retinanet.eval()

	unnormalize = UnNormalizer()

	def draw_caption(image, box, caption):
		# 绘制一个标签，下为一个大一点的黑字，上为一个小一点的白字，有一种镂空的效果
		b = np.array(box).astype(int)
		cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
		cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)

	for idx, data in enumerate(dataloader_val):

		with torch.no_grad():
			st = time.time()
			if torch.cuda.is_available():  # 输出scores，分类的类别，和框框的位置
				scores, classification, transformed_anchors = retinanet(data['img'].cuda().float())
			else:
				scores, classification, transformed_anchors = retinanet(data['img'].float())
			print('Elapsed time: {}'.format(time.time()-st))
			idxs = np.where(scores.cpu()>0.5)
			# unnormalize函数将预处理中的normalize反了过来
			img = np.array(255 * unnormalize(data['img'][0, :, :, :])).copy()

			# 限制图片的值不出现异常
			img[img<0] = 0
			img[img>255] = 255

			img = np.transpose(img, (1, 2, 0))

			# 生成RGB图片
			img = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2RGB)

			for j in range(idxs[0].shape[0]):
				bbox = transformed_anchors[idxs[0][j], :]
				x1 = int(bbox[0])
				y1 = int(bbox[1])
				x2 = int(bbox[2])
				y2 = int(bbox[3])
				label_name = dataset_val.labels[int(classification[idxs[0][j]])]
				# 绘制类别名
				draw_caption(img, (x1, y1, x2, y2), label_name)
				# 绘制框框
				cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 0, 255), thickness=2)
				print(label_name)

			cv2.imshow('img', img)
			cv2.waitKey(100)


if __name__ == '__main__':
	main()
